## Multivariate Empirical CDF Algorithm

This algorithm uses Divide and Conquer in order to calculate the multivariate empirical 
cdf, with $O(n\log_2^{(d-1)} n)$.

Algorithm based on 
Lee, D., & Joe, H. (2018). Efficient computation of multivariate empirical distribution functions at the observed values. Computational Statistics, 33(3), 1413-1428.

In [563]:
using LinearAlgebra
using Random
using CairoMakie
using Distances
CairoMakie.activate!(type = "svg")
using Debugger
using StatsBase
using BenchmarkTools
using Random

In [564]:
x = collect(1:8)
y = [8,4,3,6,5,1,2,7]*10
p = ones(8)
# p = [1,1,1,1,0,0,1,1]

N = 100
x = rand(N)
y = rand(N)
v = sortperm(x)
x = x[v]
y = y[v]
p = normalize!(rand(N),1)
bruteecdf(x,y,p) ≈ bivariatedominance(x,y,p)

true

In [565]:
function bruteecdf(x,y,p=fill(1/length(y),length(y)))
    n = length(y)
    z = zeros(n)
    for i in 1:n
        xbool = x[i] .≥ x[1:end]
        ybool = y[i] .≥ y[1:end]
        z[i] = (xbool .* ybool) ⋅ p
    end
    return z
end

bruteecdf (generic function with 2 methods)

In [566]:
function bivariatedominance(x,y,p=fill(1/length(y),length(y)))
    function Merge!(y1,y2)
        y1,y2
        N1 = length(y1)
        N2 = length(y2)
        N  = N1 + N2
        i,j,k,b = 1,1,1,0
        y = zeros(Int, N)
        while i ≤ N1 && j ≤ N2
            if y1[i] ≤ y2[j]
                y[k]  = y1[i]
                b = b+p[y[k]]
                i = i+1
            else
                y[k] = y2[j]
                z[y2[j]] = z[y2[j]] + b
                j = j+1
            end
            k = k+1
        end
        if i ≤ N1
            y[k:end] = y1[i:end]
        end

        if j ≤ N2
            y[k:end] = y2[j:end]
            z[y[k:end]] = z[y[k:end]] .+ b
        end
        return y
    end

    function Sort!(y)
        N = length(y)
        if N == 1
            return y
        else
            m = floor(Int,N/2)
            y1= Sort!(y[1:m])
            y2= Sort!(y[m+1:N])
            y = Merge!(y1,y2)
            return y
        end
    end
    
#     @assert issorted(x)
    v = sortperm(x)
    yᵣ = sortperm(y[v])
#     yᵣ = sortperm(y)
    z = zeros(length(y))
    Sort!(yᵣ)
    
    return z.+p
end

bivariatedominance (generic function with 2 methods)

In [596]:
nonunique2!(r, dims=1)

LoadError: MethodError: no method matching nonunique2!(::Matrix{Int64}; dims=1)
[0mClosest candidates are:
[0m  nonunique2!(::AbstractArray{T, N} where N) where T at In[567]:1[91m got unsupported keyword argument "dims"[39m

In [567]:
function nonunique2!(x::AbstractArray{T}) where T
    sort!(x)
    duplicatedvector = T[]
    for i=2:length(x)
        if (isequal(x[i],x[i-1]) && (length(duplicatedvector)==0 || !isequal(duplicatedvector[end], x[i])))
            push!(duplicatedvector,x[i])
        end
    end
    duplicatedvector
end

nonunique2! (generic function with 1 method)

In [602]:
Random.seed!(10)
r = rand([1,2,3,4],4,2)
d = sortslices(r,dims=1)
# d = sortslices(rand(10,2),dims=1)
v = sortperm(collect(eachrow(r)))
r[v,:]

4×2 Matrix{Int64}:
 1  2
 1  4
 4  1
 4  1

In [598]:
r

4×2 Matrix{Int64}:
 1  4
 1  2
 4  1
 4  1

In [590]:
x, y = d[:,1], d[:,2]
d

4×2 Matrix{Int64}:
 1  2
 1  4
 4  1
 4  1

In [591]:
bivariatedominance(x,y) ≈ bruteecdf(x,y)

false

In [592]:
bivariatedominance(x,y)

4-element Vector{Float64}:
 0.25
 0.5
 0.25
 0.5

In [593]:
bruteecdf(x,y)

4-element Vector{Float64}:
 0.25
 0.5
 0.5
 0.5

Sort (generic function with 1 method)

In [621]:

U = zeros(10,10)
U[1,begin]=1
for i in 1:9
    U[i+1,:] = ones(10)*i
end

In [641]:
using LinearAlgebra, BandedMatrices
L = 2
T = 1
c = 1
d = 1
Δx = 0.1
Δt = 0.1
f(a) = sin(π*a/2)^2
u⁰ = f.(0:Δx:L)
ω = c*Δt/(Δx^2)
ζ = ω + 1 - d
K = BandedMatrix(-ω*Ones(length(u⁰),length(u⁰)), (1,1))
K[BandedMatrices.band(0)] .= ζ
U = zeros(length(0:Δt:T),length(u⁰))
U[1,:] = u⁰
U[1,begin] = 0
U[1,end] = 0
for i in 1:length(0:Δt:T-Δt)
    @show i
    @show K\U[i,:]
    @show U[i+1,:] = K\U[i,:]
    U[i+1,begin] = 0
    U[i+1,end] = 0
end

# plot(collect(0:Δx:L),U[end,:])

i = 1
K \ U[i, :] = [0.008138140444159912, 0.008138140444159912, -0.002447174185242322, -0.020134464910654862, -0.03829802811078888, -0.052712713481386654, -0.06441468537059777, -0.0771528216079585, -0.0921273988519844, -0.10542542696277328, -0.11085085392554658, -0.10542542696277332, -0.0921273988519844, -0.07715282160795847, -0.06441468537059776, -0.052712713481386675, -0.03829802811078894, -0.020134464910654914, -0.0024471741852423266, 0.00813814044415995, 0.00813814044415995]
U[i + 1, :] = K \ U[i, :] = [0.008138140444159912, 0.008138140444159912, -0.002447174185242322, -0.020134464910654862, -0.03829802811078888, -0.052712713481386654, -0.06441468537059777, -0.0771528216079585, -0.0921273988519844, -0.10542542696277328, -0.11085085392554658, -0.10542542696277332, -0.0921273988519844, -0.07715282160795847, -0.06441468537059776, -0.052712713481386675, -0.03829802811078894, -0.020134464910654914, -0.0024471741852423266, 0.00813814044415995, 0.00813814044415995]
i = 2
K \ U[i, :] = [-

In [643]:
U

11×21 Matrix{Float64}:
 0.0   0.0244717    0.0954915    …   0.0954915     0.0244717   0.0
 0.0   0.00813814  -0.00244717      -0.00244717    0.00813814  0.0
 0.0  -0.00687009  -0.000813814     -0.000813814  -0.00687009  0.0
 0.0   0.00615986   0.000687009      0.000687009   0.00615986  0.0
 0.0  -0.00638224  -0.000615986     -0.000615986  -0.00638224  0.0
 0.0   0.00676008   0.000638224  …   0.000638224   0.00676008  0.0
 0.0  -0.00719539  -0.000676008     -0.000676008  -0.00719539  0.0
 0.0   0.00766222   0.000719539      0.000719539   0.00766222  0.0
 0.0  -0.00816022  -0.000766222     -0.000766222  -0.00816022  0.0
 0.0   0.00869065   0.000816022      0.000816022   0.00869065  0.0
 0.0  -0.00925558  -0.000869065  …  -0.000869065  -0.00925558  0.0